## This is a solution for Predict the Correct Grouping of Unevenly Spaced Items
We can use a Name Entity Recognition (NER) model to solve this problem. Named-entity recognition (NER) is a subtask of information extraction, that is used to identify and group tokens into a predefined set of named entities.

Here is an example of a sentence tagged using IOB tags

![img](https://miro.medium.com/v2/resize:fit:982/format:webp/1*iMV2d1OygbT6F8fX0c5YNg.png)

The IOB format (short for inside, outside, beginning) is a tagging format that is used for tagging tokens in a chunking task such as named-entity recognition. These tags are similar to part-of-speech tags but give us information about the location of the word in the chunk. The IOB Tagging system contain tags of the form:

* B-{CHUNK_TYPE} — for the word in the beginning of the chunk
* I-{CHUNK_TYPE} — for words inside the chunk
* O — for words outside/not part of the chunk

In order to use ner model to solve, we need to setup out data in such way we can fit it into the model and create label in IOB schema. Later, we can reconstruct group of each element by grouping consecutive elements

For example, given label of each elements [0, B, I, O, B, I, I], we will have 2 group (a, b) and another group (c, d), and ignore all space entity 
![Imgur](https://i.imgur.com/sgkfW8B.png)

### Setup

In [1]:
import json
import glob
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
files = glob.glob('Locofy_MLE_Challenge_Groupings/Data/*.json')
files

['Locofy_MLE_Challenge_Groupings/Data/retool.json',
 'Locofy_MLE_Challenge_Groupings/Data/wego.json',
 'Locofy_MLE_Challenge_Groupings/Data/airbnb.json',
 'Locofy_MLE_Challenge_Groupings/Data/uber.json']

### Data preparation
In the given dataset we have 3 kind of information, which will be encoded to correct format and input to LSTM model
* size
* entity type ('0' -> space otherwise enity isn't space)
* direction

In [3]:

Xs = []
ys = []

for fname in files:
    path = fname
    with open(path, 'r') as f:
        data = json.load(f)

    for elements in data:
        X = []
        y = ['O']*len(elements['input'])

        indices = {}
        # encode direction 
        direction = 0 if elements['direction'] == 'horizontal' else 1

        for idx, e in enumerate(elements['input']):
            eid = e[0]
            
            # encode entity type
            if eid == '0':
                etype = 0
            else:
                etype = 1
                indices[eid] = idx
            
            # size
            sz = e[1]
            X.append((etype, sz))

        
        # label encode
        start_index = []
        end_index = []
        for entity in elements['output']:
            if len(entity) > 1:
                start_index.append(indices[entity[0]])
                end_index.append(indices[entity[-1]])
                if ord(entity[0]) >= ord(entity[-1]):
                    print(entity[0], entity[-1])
                    print('lol')
        
        # using BIO schema 
        for start, end in zip(start_index, end_index):
            y[start:end+1] = ['B-TAG'] + ['I-TAG']*(end-start)
        
        X = {'elements': X, 'direction': direction}
        Xs.append(X)
        ys.append(y)

In [4]:
Xs[-1]

{'elements': [(1, 36), (1, 24), (0, 15), (1, 24), (1, 0)], 'direction': 1}

In [5]:
ys[-1]

['O', 'B-TAG', 'I-TAG', 'I-TAG', 'O']

### Dataloader
Create batch of sample by right padding all sequence to max_length_sequence in list, and clip entity size in range [0, 1023]


In [6]:
from torch.utils.data import Dataset, DataLoader
import torch
from torch.nn.utils.rnn import pad_sequence

def get_mask(batch_tensor):
    mask = batch_tensor.eq(0)
    mask = mask.eq(0)
    return mask

def my_collate_fn(batch):
    batch = {key: [d[key] for d in batch] for key in batch[0]}
    
    tag, seq_length = my_collate(batch['tag'])
    sz = my_collate(batch['sz'])[0]
    label = my_collate(batch['label'])[0]
    direction = torch.tensor(batch['direction'])
    mask = get_mask(label)
    
    return tag, sz, seq_length, direction, mask, label

def my_collate(batch_tensor):
    word_seq_lengths = torch.LongTensor(list(map(len, batch_tensor)))
    batch_tensor.sort(key=lambda x: len(x), reverse=True)
    tensor_length = torch.tensor([len(sq) for sq in batch_tensor])
    batch_tensor = pad_sequence(batch_tensor, batch_first=True, padding_value=0)
    
    return batch_tensor, tensor_length
    
class MyDataset(Dataset):
    def __init__(self, Xs, ys, aug=False):

        self.Xs = Xs
        self.ys = ys
        self.labels = ['pad', 'O', 'B-TAG', 'I-TAG']
        self.label2idx = {v:k for k, v in enumerate(self.labels)}
        self.aug = aug
        
    def __len__(self):
        return len(self.Xs)

    def __getitem__(self, item):
        
        X = self.Xs[item]['elements']
        y = self.ys[item]
        
        X = torch.tensor(X)
        y = torch.tensor([self.label2idx[i] for i in y])
        
        direction = torch.tensor(self.Xs[item]['direction'])
        tag = X[:, 0]
        
        delta = 1.0
        if self.aug and np.random.uniform(0, 1) > 0.8: 
            delta = np.random.uniform(0.8, 1.2)
#         print(delta)
        # max size of entity is 1023
        sz = (X[:, 1]*delta).int()
        sz  = torch.clip(sz, 0, 1023)
        
        x = torch.random
        return {'direction': direction, 'tag': tag, 'sz': sz, 'label': y}

X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.1)

train_dataset = MyDataset(X_train, y_train, aug=True)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=my_collate_fn)

test_dataset = MyDataset(X_test, y_test, aug=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=my_collate_fn)

Dataloader will return **tag, sz, seq_length, direction, mask, label** 

In [7]:
next(iter(train_loader))

(tensor([[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
         [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0],
         [0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0],
         [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0]]),
 tensor([[  8,  12,   8,  10,   8,  10,   8,  10,   8,  10,   8,  10,   8,  10,
           32,  23,   8],
         [115, 141,   7,  23,   9,  34,   4,  52, 175, 115,  82,   7, 121,  24,
           21,   7,   0],
         [ 64,  51,  24, 117,  70,  58,   4, 597,  24,  75, 114,   4,  68,  74,
           64,   0,   0],
         [392, 100,  16,  54,  16,  46,  16,  62,  16,  20,  16,  20,  16,  16,
          408,   0,   0]], dtype=torch.int32),
 tensor([17, 16, 15, 15]),
 tensor([0, 0, 0, 0]),
 tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  Tru

### Create NER model
The architecture of ner model is shown in bellow. it contains some components
* Entity type embedding
* Size embedding
* Direction embedding
* Directional LSTM
* Classifer

Entity embedded and size embedded will be concatenated, and fetch into LSTM. Direction embedded will be initialized for initial hidden states. We predict tag of element at each timestep

We train using crossentropy loss
<img src="https://i.imgur.com/2ClkyI7.png" alt="drawing" width="800"/>


In [8]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import numpy as np

class NamedEntityRecog(nn.Module):
    def __init__(self, tag_size, direction_num, max_postion, tag_embed_dim, size_embed_dim, direction_embed_dim, hidden_dim, label_num, dropout=0.1):
        super(NamedEntityRecog, self).__init__()

        self.drop = nn.Dropout(dropout)
        self.tag_embeds = nn.Embedding(tag_size, tag_embed_dim, padding_idx=0)
        self.size_embeds = nn.Embedding(max_postion, size_embed_dim, padding_idx=0)
        self.direction_embeds = nn.Embedding(direction_num, direction_embed_dim, padding_idx=0)
        
        self.input_dim = tag_embed_dim + size_embed_dim
        self.lstm = nn.LSTM(self.input_dim, hidden_dim, batch_first=True, bidirectional=True)

        self.hidden2tag = nn.Linear(hidden_dim * 2 , label_num)


    def forward(self, tag, sz, seq_length, direction, mask, label=None):        
        batch_size = tag.size(0)
        seq_len = tag.size(1)
        
        tag_embedding = self.tag_embeds(tag)
        sz_embedding = self.size_embeds(sz)
        dir_embedding = self.direction_embeds(direction)
        word_embeding = torch.cat([tag_embedding, sz_embedding], 2)
        
        word_represents = self.drop(word_embeding)
        
        packed_words = pack_padded_sequence(word_represents, seq_length, True)
        hidden = dir_embedding[None].expand(2, -1, -1)
        
        lstm_out, hidden = self.lstm(packed_words, (hidden, hidden))
        lstm_out, _ = pad_packed_sequence(lstm_out)
        lstm_out = lstm_out.transpose(0, 1)
        feature_out = self.drop(lstm_out)
        feature_out = self.hidden2tag(feature_out)
        
        if label != None:
            loss_function = nn.CrossEntropyLoss(ignore_index=0, reduction='sum')

            feature_out = feature_out.contiguous().view(batch_size * seq_len, -1)
            total_loss = loss_function(feature_out, label.contiguous().view(batch_size * seq_len))

            return total_loss
        else:
            feature_out = feature_out.contiguous().view(batch_size * seq_len, -1)
            _, tag_seq = torch.max(feature_out, 1)
            tag_seq = tag_seq.view(batch_size, seq_len)
            tag_seq = mask.long() * tag_seq
            return tag_seq       


### Evaluate
Given predict labels and true labels, we compute precision, recall, f1 per entity by using seqeval lib. 

Here is an example predicted tags and labels 
 
y_true = [['O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
 
y_pred = [['O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
 


In [9]:
from seqeval.metrics import f1_score, precision_score, recall_score

tag2label = train_dataset.labels

def evaluate(dataloader, model):
    model.eval()
    prediction = []
    
    gt_list = []
    pred_list = []
    for batch in dataloader:
        tag, sz, seq_length, direction, mask, label = batch

        tag_seq = model(tag, sz, seq_length, direction, mask)
        for i in range(len(tag)):
            gt = [tag2label[tag_idx] for idx, tag_idx in enumerate(label[i]) if mask[i][idx]==True]
            pred = [tag2label[tag_idx] for idx, tag_idx in enumerate(tag_seq[i]) if mask[i][idx]==True]

            assert len(gt) == len(pred)
            gt_list.append(gt)
            pred_list.append(pred)
            
            
    metrics = {
            "precision": precision_score(gt_list, pred_list),
            "recall": recall_score(gt_list, pred_list),
            "hmean": f1_score(gt_list, pred_list),
        }
    
    return metrics

### Train and evaluate model performance

In [10]:
import torch.optim as optim

model = NamedEntityRecog(
    tag_size=2, 
    direction_num=2,
    max_postion=1024, 
    tag_embed_dim=16, 
    size_embed_dim=8, 
    direction_embed_dim=32,
    hidden_dim=32, 
    label_num=4)

optimizer = optim.Adam(model.parameters(), lr=0.001)
model

NamedEntityRecog(
  (drop): Dropout(p=0.1, inplace=False)
  (tag_embeds): Embedding(2, 16, padding_idx=0)
  (size_embeds): Embedding(1024, 8, padding_idx=0)
  (direction_embeds): Embedding(2, 32, padding_idx=0)
  (lstm): LSTM(24, 32, batch_first=True, bidirectional=True)
  (hidden2tag): Linear(in_features=64, out_features=4, bias=True)
)

In [11]:
from torch.nn.utils import clip_grad_norm_

best_f1 = 0
for epoch in range(50):
    model.train()

    for batch in train_loader:
        model.zero_grad()
        tag, sz, seq_length, direction, mask, label = batch

        loss = model(tag, sz, seq_length, direction, mask, label)
        loss.backward()
        clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
    
    metrics = evaluate(test_loader, model)
    
    if metrics['hmean'] > best_f1:
        best_f1 = metrics['hmean']
        torch.save(model.state_dict(), 'model.pt')
    print(metrics)
    
print('model save to model.pt and best h1 : {}'.format(best_f1))

{'precision': 0.4348958333333333, 'recall': 0.5585284280936454, 'hmean': 0.4890190336749633}
{'precision': 0.767515923566879, 'recall': 0.8060200668896321, 'hmean': 0.7862969004893965}
{'precision': 0.8557377049180328, 'recall': 0.8729096989966555, 'hmean': 0.8642384105960265}
{'precision': 0.8737864077669902, 'recall': 0.903010033444816, 'hmean': 0.888157894736842}
{'precision': 0.896774193548387, 'recall': 0.9297658862876255, 'hmean': 0.9129720853858785}
{'precision': 0.9120521172638436, 'recall': 0.9364548494983278, 'hmean': 0.9240924092409241}
{'precision': 0.9177631578947368, 'recall': 0.9331103678929766, 'hmean': 0.9253731343283582}
{'precision': 0.9570957095709571, 'recall': 0.9698996655518395, 'hmean': 0.9634551495016612}
{'precision': 0.9601328903654485, 'recall': 0.9665551839464883, 'hmean': 0.9633333333333334}
{'precision': 0.9603960396039604, 'recall': 0.9732441471571907, 'hmean': 0.9667774086378738}
{'precision': 0.9766666666666667, 'recall': 0.979933110367893, 'hmean': 0.

f1 measure is high so model performance is very good 

### Suggested improvement
With the current model performance and the current dataset, i think we already solved the problem well, no need to enhance anymore. anw, i still list some suggestions for further improvement 
* add entity dimension: topleft, width, height
* entity image
* text inside entity

Add image and text into model will increase complexity and slow down inference 

### Export to onnx for deployment

In [12]:
import torch.onnx

input_names = ['tag', 'sz', 'seq_length', 'direction', 'mask']
output_names = ['output']
dummy = tag, sz, seq_length, direction, mask
dummy = tuple(e[[0]] for e in dummy)
input_shape = {
                'tag': {0: 'batch_size', 1:'seq_length'},
                'sz': {0: 'batch_size', 1:'seq_length'},
                'seq_length': {0: 'batch_size'},
                'direction': {0:'batch_size'},
                'mask':{0:'batch_size', 1:'seq_length'}
              }

state_dict = torch.load('model.pt')
model.load_state_dict(state_dict)

torch.onnx.export(model, dummy, 'model.onnx', input_names=input_names, output_names=output_names, 
                 dynamic_axes={**input_shape,   
                                'output' : {0 : 'batch_size', 1:'seq_length'}})

[W shape_type_inference.cpp:1920] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)


============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 4 WARNING 0 ERROR ========================
4 WARNING were not printed due to the log level.



[W shape_type_inference.cpp:1920] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
/home/pbcquoc/anaconda3/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:4476: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
[W shape_type_inference.cpp:1920] Warning: The shape inference of prim::PadPacked type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W shape_type_inference.cpp:1920] Warning: The shape inference of prim::PadPacked type is missing, so it may result in wro

### Testing my model using API
I also provide endpoint for testing my model. 
Here is an example using curl command
```
curl --header "Content-Type: application/json" --request POST --data '{"input": [["0", 64], ["a", 51], ["0", 24], ["b", 117], ["c", 70], ["d", 58], ["e", 4], ["0", 597], ["f", 24], ["g", 75], ["h", 114], ["i", 4], ["j", 68], ["k", 74], ["0", 64]], "direction": "horizontal"}' https://api.vocr.vn/predictions/ner
```

In [15]:
!curl --header "Content-Type: application/json" --request POST --data '{"input": [["0", 64], ["a", 51], ["0", 24], ["b", 117], ["c", 70], ["d", 58], ["e", 4], ["0", 597], ["f", 24], ["g", 75], ["h", 114], ["i", 4], ["j", 68], ["k", 74], ["0", 64]], "direction": "horizontal"}' https://api.vocr.vn/predictions/ner

{
  "output": [
    "a",
    [
      "b",
      "c",
      "d",
      "e"
    ],
    "f",
    [
      "g",
      "h",
      "i",
      "j",
      "k"
    ]
  ]
}

In [17]:
import requests
def post(x):
    data = requests.post('https://api.vocr.vn/predictions/ner', 
                         json={ 'input':x['input'], 'direction':x['direction']})
    return data.json()

x = {"input": [["0", 64], ["a", 51], ["0", 24], ["b", 117], ["c", 70], ["d", 58], ["e", 4], ["0", 597], ["f", 24], ["g", 75], ["h", 114], ["i", 4], ["j", 68], ["k", 74], ["0", 64]], 
     "direction": "horizontal"}
post(x)

{'output': ['a', ['b', 'c', 'd', 'e'], 'f', ['g', 'h', 'i', 'j', 'k']]}